<a href="https://colab.research.google.com/github/harry934/MACHINE-LEARNING-PROJECTS/blob/main/EmergencyResponce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ================================================
# SMART ACCIDENT PREDICTION SYSTEM (KENYA)
# CREATE DATASET + TRAIN NEURAL NETWORK + MAP
# ================================================

import numpy as np
import pandas as pd
import random
from datetime import datetime, timedelta
import tensorflow as tf
from tensorflow.keras import layers, models
import folium
from folium.plugins import HeatMap

# ---------------------------
# STEP 1 — CREATE YOUR OWN DATASET
# ---------------------------

counties = {
    "Nairobi": (-1.286389, 36.817223),
    "Mombasa": (-4.043477, 39.668206),
    "Kisumu": (-0.091702, 34.7680),
    "Nakuru": (-0.3031, 36.0800),
    "Eldoret": (0.5143, 35.2699),
}

weather_options = ["Clear", "Rainy", "Fog", "Sunny", "Storm"]
road_types = ["Highway", "Urban", "Rural"]
vehicle_types = ["Car", "Motorbike", "Truck", "Bus"]

def random_date():
    start = datetime(2022,1,1)
    end = datetime(2025,11,1)
    delta = end - start
    return start + timedelta(seconds=random.randint(0,int(delta.total_seconds())))

records = []
for _ in range(5000):
    county, (latc, lonc) = random.choice(list(counties.items()))
    lat = latc + np.random.uniform(-0.02, 0.02)
    lon = lonc + np.random.uniform(-0.02, 0.02)
    dt = random_date()
    hour = dt.hour
    weather = random.choice(weather_options)
    road = random.choice(road_types)
    vehicle = random.choice(vehicle_types)

    # Accident risk score (label)
    risk = 0.2
    if weather == "Rainy": risk += 0.2
    if road == "Highway": risk += 0.1
    if vehicle == "Motorbike": risk += 0.1
    if hour >= 20 or hour <= 5: risk += 0.15

    risk = min(1, risk)

    records.append([dt.date(), dt.time(), county, lat, lon, weather, road, vehicle, hour, risk])

df = pd.DataFrame(records, columns=[
    "Date","Time","County","Latitude","Longitude","Weather","Road_Type",
    "Vehicle_Type","Hour","Risk"
])

df.to_csv("kenya_accidents_synthetic.csv", index=False)
df.head()


In [ ]:
# ---------------------------
# STEP 2 — DATA PREPARATION
# ---------------------------

df_nn = df.copy()

# Convert categorical → numeric
df_nn = pd.get_dummies(df_nn, columns=["County","Weather","Road_Type","Vehicle_Type"], drop_first=True)

X = df_nn.drop(["Date","Time","Risk"], axis=1)
y = df_nn["Risk"]

# Normalize
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# ---------------------------
# STEP 3 — NEURAL NETWORK MODEL
# ---------------------------

model = models.Sequential([
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # risk = probability
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

# Train
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)


In [ ]:
# ---------------------------
# STEP 4 — PREDICT
# ---------------------------

y_pred = model.predict(X_test)
print("Sample Predictions:", y_pred[:10])


In [ ]:
# ---------------------------
# STEP 5 — HEATMAP OF ACCIDENT HOTSPOTS
# ---------------------------

m = folium.Map(location=[0.1, 37.0], zoom_start=6)

heat_data = df[['Latitude','Longitude']].values.tolist()

HeatMap(heat_data, radius=8).add_to(m)

m.save("kenya_accident_heatmap.html")
m
